# III B. Building a model to fit the Mulliken Charges

There are several files with additional features, which have information only for train and not for test. This kernel is to use the Mulliken charges data, which is provided only for train dataset, and predict it for the test set to be used as a feature in the final model.

In [1]:
import pandas as pd
import numpy as np
import time, copy
import matplotlib.pyplot as plt
import os

In [2]:
file_folder = '../input/champs-scalar-coupling'
mulliken = pd.read_csv(f'{file_folder}/mulliken_charges.csv')

In [3]:
acsf_folder = '../input/acsfstructures'
acsf_structures = pd.read_pickle(f'{acsf_folder}/acsf.pkl')

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# print(train.shape, test.shape)#, df_train_sub_charge.shape, df_train_sub_tensor.shape)

df_acsf = reduce_mem_usage(acsf_structures)

df_mulliken = reduce_mem_usage(mulliken)

Mem. usage decreased to 515.11 Mb (73.7% reduction)
Mem. usage decreased to 16.09 Mb (54.2% reduction)


In [5]:
# Create train and test sets from the structure file containing ACSF
mulliken_train = df_acsf[df_acsf['molecule_name'].isin(mulliken['molecule_name'].unique())]
mulliken_train.reset_index(drop=True, inplace=True)
# Rest is test dataset
mulliken_test = df_acsf[~df_acsf['molecule_name'].isin(mulliken['molecule_name'].unique())]

In [6]:
# Sanity check for dataset size
print( len(mulliken_train), len(mulliken) )
print( len(mulliken_test), len(df_acsf) - len(mulliken) )

1533537 1533537
825120 825120


In [7]:
# Remove the feature not required
mulliken_charges_train = mulliken_train.drop(['molecule_name', 'atom_index'], axis=1)#, inplace=True)
mulliken_charges_test = mulliken_test.drop(['molecule_name', 'atom_index'], axis=1)#, inplace=True)
mulliken_charges_train.head()

,atom,g1,g2_2_0.01,g4_2_0.01_1_1,g4_2_0.01_1_8,g4_2_0.01_1_16,g4_2_0.01_-1_1,g4_2_0.01_-1_8,g4_2_0.01_-1_16,g2_6_0.01,...,g4_1_0.2_0.5_1,g4_1_0.2_0.5_8,g4_1_0.2_0.5_16,g2_1.5_0.2,g4_1.5_0.2_1_1,g4_1.5_0.2_1_8,g4_1.5_0.2_1_16,g4_1.5_0.2_0.5_1,g4_1.5_0.2_0.5_8,g4_1.5_0.2_0.5_16
0,C,2.021484,2.003906,0.173340,0.000079,0.000000,0.346680,2.027893e-02,0.000791,1.588867,...,0.194336,0.000424,3.576279e-07,1.955078,0.162354,0.000074,0.000000,0.202881,0.000443,4.172325e-07
1,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,0.000000,0.613281,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04
2,H,0.763672,0.759277,0.241821,0.121155,0.055847,0.025757,1.192093e-07,0.000000,0.613281,...,0.167480,0.014214,8.544922e-04,0.743164,0.226562,0.113464,0.052307,0.176025,0.014885,8.931160e-04
3,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,0.000000,0.613281,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04
4,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,0.000000,0.613281,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04


In [8]:
one_hot_encoded_training_predictors = pd.get_dummies(mulliken_charges_train)
one_hot_encoded_testing_predictors = pd.get_dummies(mulliken_charges_test)

mulliken_charges_train, mulliken_charges_test = one_hot_encoded_training_predictors.align(one_hot_encoded_testing_predictors,
                                                                    join='left', 
                                                                    axis=1)
mulliken_charges_test.head()

,g1,g2_2_0.01,g4_2_0.01_1_1,g4_2_0.01_1_8,g4_2_0.01_1_16,g4_2_0.01_-1_1,g4_2_0.01_-1_8,g4_2_0.01_-1_16,g2_6_0.01,g4_6_0.01_1_1,...,g4_1.5_0.2_1_8,g4_1.5_0.2_1_16,g4_1.5_0.2_0.5_1,g4_1.5_0.2_0.5_8,g4_1.5_0.2_0.5_16,atom_C,atom_F,atom_H,atom_N,atom_O
12,0.956543,0.949219,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.753906,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,1,0,0,0,0
13,0.956543,0.949219,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.753906,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,1,0,0,0,0
14,0.526855,0.522461,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.412842,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0,0,1,0,0
15,0.526855,0.522461,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.412842,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0,0,1,0,0
79,1.789062,1.776367,0.09198,0.000049,0.0,0.177979,0.009666,0.000348,1.414062,0.048553,...,0.000046,0.0,0.106323,0.000247,2.384186e-07,1,0,0,0,0


In [9]:
# Add Mulliken charges to training set
mulliken_charges_train['mulliken_charge'] = df_mulliken['mulliken_charge']

In [10]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error

target = 'mulliken_charge'

X_train = mulliken_charges_train.drop(target, axis=1)    
y_train = mulliken_charges_train[target]

# Extra Tree
# Hyperparameters obtained from iterations that gave back least mean squared error
# Here, I'm using the best performing Hyperparameters out of them
reg = ExtraTreesRegressor(n_estimators=8, max_depth=20, n_jobs=4)
reg.fit(X_train, y_train)
pred_train = reg.predict(X_train)

print('MAE on train set: %.2E.' %mean_absolute_error(y_train, pred_train)) 

MAE on train set: 1.62E-02.


In [11]:
mulliken_predict_test = reg.predict(mulliken_charges_test)

# Trimming out not required columns from test dataset
cols = [col for col in mulliken_test.columns[2:] ]

Now, we create the test file's Mulliken charges like we had for the train molecules, thereby dropping cols we added

In [12]:
mulliken_test.drop(columns=cols, inplace=True)
mulliken_test['mulliken_charge'] = mulliken_predict_test
mulliken_test.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,molecule_name,atom_index,mulliken_charge
12,dsgdb9nsd_000004,0,-0.420117
13,dsgdb9nsd_000004,1,-0.420117
14,dsgdb9nsd_000004,2,0.200479
15,dsgdb9nsd_000004,3,0.200479
79,dsgdb9nsd_000015,0,-0.224901


In [13]:
mulliken_train.head()

,molecule_name,atom_index,atom,g1,g2_2_0.01,g4_2_0.01_1_1,g4_2_0.01_1_8,g4_2_0.01_1_16,g4_2_0.01_-1_1,g4_2_0.01_-1_8,...,g4_1_0.2_0.5_1,g4_1_0.2_0.5_8,g4_1_0.2_0.5_16,g2_1.5_0.2,g4_1.5_0.2_1_1,g4_1.5_0.2_1_8,g4_1.5_0.2_1_16,g4_1.5_0.2_0.5_1,g4_1.5_0.2_0.5_8,g4_1.5_0.2_0.5_16
0,dsgdb9nsd_000001,0,C,2.021484,2.003906,0.173340,0.000079,0.000000,0.346680,2.027893e-02,...,0.194336,0.000424,3.576279e-07,1.955078,0.162354,0.000074,0.000000,0.202881,0.000443,4.172325e-07
1,dsgdb9nsd_000001,1,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04
2,dsgdb9nsd_000001,2,H,0.763672,0.759277,0.241821,0.121155,0.055847,0.025757,1.192093e-07,...,0.167480,0.014214,8.544922e-04,0.743164,0.226562,0.113464,0.052307,0.176025,0.014885,8.931160e-04
3,dsgdb9nsd_000001,3,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04
4,dsgdb9nsd_000001,4,H,0.763672,0.759277,0.241821,0.121094,0.055786,0.025772,1.192093e-07,...,0.167480,0.014206,8.535385e-04,0.743164,0.226562,0.113403,0.052216,0.176025,0.014877,8.921623e-04


The Mulliken test data is organized to have columns in order simlar to train set

In [15]:
mulliken_test.head()

,molecule_name,atom_index,mulliken_charge
12,dsgdb9nsd_000004,0,-0.420117
13,dsgdb9nsd_000004,1,-0.420117
14,dsgdb9nsd_000004,2,0.200479
15,dsgdb9nsd_000004,3,0.200479
79,dsgdb9nsd_000015,0,-0.224901


In [14]:
mulliken_test.to_pickle('mulliken_test.pkl')